# Humpback Whale Identification - sklearn SVM + B&W

In [1]:
import gc
import os

import numpy as np
import pandas as pd
import progressbar

import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, svm

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv("../input/train.csv")
train_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [3]:
# yvan made this block, he's really proud of him
import cv2

def colors2BlackAndWhite(path):
    #-----Reading the image-----------------------------------------------------
    img = cv2.imread(path)
    #-----Black and White-------------------------------------------------------
    bw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    #limg = cv2.merge((bw, bw, bw)) # (cl, a, b)
    #-----Converting image from LAB Color model to RGB model--------------------
    final = bw #final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    #-----Resize----------------------------------------------------------------
    resized = cv2.resize(final, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    return resized

In [ ]:
def prepareImages(data, m, dataset):
    #print("Preparing images")
    #print(m)
    X_train = np.zeros((m, 128 * 128))
    count = 0
    
    for fig in progressbar.progressbar(data['Image']):
        # load images into images of size 128x128
        x = colors2BlackAndWhite("../input/"+dataset+"/"+fig) # added by Yvan
        x = preprocess_input(x)
        X_train[count] = x.flatten() # <------------------ I FIXED A BUG HERE
        count += 1
    
    return X_train

In [ ]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    #print(integer_encoded)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    #print(onehot_encoded)

    y = onehot_encoded
    #print(y.shape)
    return y, label_encoder

In [ ]:
X = prepareImages(train_df, train_df.shape[0], "train")
X /= 255

100% (25361 of 25361) |##################| Elapsed Time: 0:02:12 Time:  0:02:12


In [ ]:
X.shape

(25361, 16384)

In [ ]:
#y, label_encoder = prepare_labels(train_df['Id'])
y = train_df['Id'].values

In [ ]:
y.shape

(25361,)

# Change model starting from here

## Train

In [ ]:
clf = svm.SVC(gamma='scale')
clf.fit(X, y)  #bloque au bout de 3heures ....

## Test

In [ ]:
test = os.listdir("../input/test/")
print(len(test))

In [ ]:
test_df = pd.DataFrame(test, columns=['Image'])
test_df['Id'] = ''

In [ ]:
X = prepareImages(test_df, test_df.shape[0], 'test')
X /= 255

In [ ]:
predictions = clf.predict(np.array(X))

In [ ]:
for i, pred in enumerate(predictions): # TODO !!!!! <------------------------------------------
    test_df.loc[i, 'Id'] = ' '.join(label_encoder.inverse_transform(pred.argsort()[-5:][::-1]))

In [ ]:
test_df.head(10)
test_df.to_csv('sklearn-SVM-black-and-white.csv', index=False) #> Score = ?

In [ ]:
!kaggle competitions submit -c humpback-whale-identification -f "sklearn-SVM-black-and-white.csv" -m "sklearn SVM + B&W"